# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
# Для работы с данными
import pandas as pd
import numpy as np
from scipy import stats as st 

# Для обучения модели
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Для оценки модели
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

## Загрузка и подготовка данных

Нахожу нецелесообразным объединять регионы, так как обучать модель и проверять ее будем отдельно на каждом.

In [3]:
# Загрузим данные в словарь, ключем же будет индекс региона
data = {}

data[0] = pd.read_csv('/datasets/geo_data_0.csv')
data[1] = pd.read_csv('/datasets/geo_data_1.csv')
data[2] = pd.read_csv('/datasets/geo_data_2.csv')

### Изучение данных

In [4]:
for region in data:
    display(data[region].head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Есть столбец *id*. Он не понадобится при обучении модели, потому впоследствии избавимся от него.

In [5]:
for region in data:
    data[region].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

Не похоже, что есть очевидные пропуски.

In [6]:
data[0].describe().T

,count,mean,std,min,25%,50%,75%,max
f0,100000.0,0.500419,0.871832,-1.408605,-0.072580,0.502360,1.073581,2.362331
f1,100000.0,0.250143,0.504433,-0.848218,-0.200881,0.250252,0.700646,1.343769
f2,100000.0,2.502647,3.248248,-12.088328,0.287748,2.515969,4.715088,16.003790
product,100000.0,92.500000,44.288691,0.000000,56.497507,91.849972,128.564089,185.364347


In [7]:
for region in data:
    print(data[region].shape[0] - data[region].drop_duplicates().shape[0], end='\t')

0	0	0	

In [8]:
for region in data:
    print(data[region][data[region].duplicated(subset=['id'])].shape[0], end='\t')

10	4	4	

Очевидных дубликатов тоже не обнаружено, дубликатов по *id* мало, потому не оставим как есть.

Создадим дублирующий датафрейм, но без id.

In [9]:
train_data = {}
for region in data:
    train_data[region] = data[region].drop(['id'], axis=1)

In [10]:
for region in data:
    display(train_data[region].head(1))

,f0,f1,f2,product
0,0.705745,-0.497823,1.22117,105.280062


,f0,f1,f2,product
0,-15.001348,-8.276,-0.005876,3.179103


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673


### Разделение данных

In [11]:
target = {}
features = {}

for region in train_data:
    target[region] = train_data[region]['product']
    features[region] = train_data[region].drop(['product'], axis=1)

In [12]:
target[0].head()

0    105.280062
1     73.037750
2     85.265647
3    168.620776
4    154.036647
Name: product, dtype: float64

In [13]:
features[0].head()

,f0,f1,f2
0,0.705745,-0.497823,1.221170
1,1.334711,-0.340164,4.365080
2,1.022732,0.151990,1.419926
3,-0.032172,0.139033,2.978566
4,1.988431,0.155413,4.751769


Данные будем делить 3 к 1.

In [14]:
target_train = {}
features_train = {}

target_valid = {}
features_valid = {}

# Просто пройдемся по регионам и для каждого разделим
for region in train_data:
    features_train[region], features_valid[region], target_train[region], target_valid[region] = (
        train_test_split(features[region], target[region], test_size=0.25, random_state=12345, shuffle = True)
    )

In [15]:
target_train[0].shape, target_valid[0].shape

((75000,), (25000,))

### Промежуточный итог

Изучили данные. Пропусков не найдено. Полных дубликатов нет. Дубликатов по *id* почти нет, потому было решено их оставить. Сам столбец *id* лишь помешает обучению модели, потому от него мы избавились. Данные  разделили для каждого региона на валидационный датасет и тренировочный(75:25).

## Обучение и проверка модели

Обучать мы будем для каждого региона, потому добавим функцию для удобного обучения.

In [16]:
def lin_model(features, target):
    model = LinearRegression()
    model.fit(features, target)
    
    return model

In [17]:
# Для оценки работы модели добавим параметры.
def param(target, predict):
    print('Средний предсказанный запас сырья:', predict.mean())
    print('Средний реальный запас сырья:', target.mean())
    print('MSE:', mean_squared_error(target, predict))
    print('RMSE:', mean_squared_error(target, predict) ** 0.5)
    print('R2:', r2_score(target, predict))
    print('\n')

### Обучение и тестирование модели

In [18]:
prediction = {}
for region in range(3):
    print(f'Регион - {region}')
    model = lin_model(features_train[region], target_train[region])
    prediction[region] = pd.Series(model.predict(features_valid[region]), index=target_valid[region].index)
    
    param(target_valid[region], prediction[region])

Регион - 0
Средний предсказанный запас сырья: 92.59256778438035
Средний реальный запас сырья: 92.07859674082927
MSE: 1412.2129364399243
RMSE: 37.5794217150813
R2: 0.27994321524487786


Регион - 1
Средний предсказанный запас сырья: 68.728546895446
Средний реальный запас сырья: 68.72313602435997
MSE: 0.7976263360391157
RMSE: 0.893099286775617
R2: 0.9996233978805127


Регион - 2
Средний предсказанный запас сырья: 94.96504596800489
Средний реальный запас сырья: 94.88423280885438
MSE: 1602.3775813236196
RMSE: 40.02970873393434
R2: 0.20524758386040443




### Промежуточный вывод

**0** и **2** регионы показывают примерно одинаковые результаты, но вот **1** регион показывает самые лучшие показатели (как модель), похоже параметр или параметры почти напрямую коррелировали с запасом сырья.

На первый взгляд по средним запасам самой перспективным выглядит **2-й регион**.

## Подготовка к расчёту прибыли

### Прибыльность

Наша цель исследовать 500 точек, из которых надо выбрать 200 лучших для разработки. Бюджет для разработки 10 млрд рублей. Один баррель стоит 450 рублей. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [19]:
BUDGET_OF_REGION = 10 * 10**9
PRICE = 450 * 10**3
BEST_SPOT = 200

In [20]:
print('Объем сырья для безубыточной разработки скважины:', BUDGET_OF_REGION / PRICE / BEST_SPOT)

Объем сырья для безубыточной разработки скважины: 111.11111111111111


Сравним со средним объемом сырья по регионам.

In [21]:
print('Средний объем запасов скважин в 0-м регионе: ', target[0].mean())
print('Средний объем запасов скважин в 1-м регионе: ', target[1].mean())
print('Средний объем запасов скважин в 2-м регионе: ', target[2].mean())

Средний объем запасов скважин в 0-м регионе:  92.50000000000001
Средний объем запасов скважин в 1-м регионе:  68.82500000000002
Средний объем запасов скважин в 2-м регионе:  95.00000000000004


### Промежуточный вывод

Средний объем значительно ниже во всех регионах, чем необходимое кол-во для безубыточной разработки.

Интересно, что в **1** регионе гораздо ниже средний объем запасов скважин, чем 111. Однако в этом именно по этому региону наша модель прогнозирует лучше всего.


## Расчёт прибыли и рисков 

In [22]:
# Прибыль для выбранных скважин
def revenue(target, predictions, cost, count):
    probs_sorted = predictions.sort_values(ascending=False)
    deposits = target[probs_sorted.index][:count]
    return cost * deposits.sum() - BUDGET_OF_REGION

In [23]:
def bootstrap(target, predictions):
    state = np.random.RandomState(12345)

    values = []
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
    
        values.append(revenue(target_subsample, probs_subsample, PRICE, 200))

    values = pd.Series(values)
    return values 

### Расчет прибыли

In [24]:
distribution = {}

for region in range(3):
    distribution[region] = bootstrap(target_valid[region], prediction[region])

In [25]:
# Сюда будем записывать распределение по регионам
results = pd.DataFrame(columns=['Регион', 'Средняя прибыль', '95%-й Доверительный интервал', 'Риск убытков'])

Будем проходится *Бутстрепом* по регионам и записывать результаты.

In [26]:
for region in range(3):
    results.loc[region, 'Регион'] =  f'{region}-й регион'
    results.loc[region, 'Средняя прибыль'] = round(distribution[region].mean())
    results.loc[region, '95%-й Доверительный интервал'] = (
        f'{round(distribution[region].quantile(.025))} - {round(distribution[region].quantile(.975))}')
    results.loc[region, 'Риск убытков'] = f'{st.percentileofscore(distribution[region], 0)}%'
    

In [27]:
results

,Регион,Средняя прибыль,95%-й Доверительный интервал,Риск убытков
0,0-й регион,425938527,-102090095 - 947976353,6.0%
1,1-й регион,515222773,68873225 - 931547591,1.0%
2,2-й регион,435008363,-128880547 - 969706954,6.4%


### Промежуточный вывод

По итогу 1-й регион самый перспективный регион по цифрам.

## Вывод

# Подготовка данных

Данные сразу не пришлось сильно предобрабатывать. Не оказалось дубликатов или ошибок. 

# Обучение и проверка модели

Исходя из значений RMSE, наилучшей выглядит модель для региона 1 со значением **0.89**

Наиболее прибыльным же регионом выглядел под номером 2 - **94.8**.

# Расчет прибыли и рисков

Наименьший риск убытка после анализа был в регионе 1 (1%), что показали и средняя прибыль, и доверительный интервал. 

1 регион -  самый стабильный и вариант. Даже при выборе не самых удачных скважин, все окупится.